In [1]:
import boto3
import pandas as pd
import numpy as np
import psycopg2
import configparser

## Leemos archivos de configuración

In [2]:
config = configparser.ConfigParser()
config.read('config.cfg')

['config.cfg']

In [3]:
aws_rds_conn = boto3.client('rds', aws_access_key_id = config.get('IAM', 'ACCES_KEY'), 
                              aws_secret_access_key = config.get('IAM', 'SECRET_ACCES_KEY'),
                              region_name = 'us-east-1')

In [4]:
rds_instances_ids = []
aws_response = aws_rds_conn.describe_db_instances()

for response in aws_response['DBInstances']:
    rds_instances_ids.append(response['DBInstanceIdentifier'])

print('Instancias Disponibles: {rds_instances_ids}')

Instancias Disponibles: {rds_instances_ids}


### Creamos instancia de base de datos en AWS - Postgres

In [5]:
try:
    response = aws_rds_conn.create_db_instance(
            DBInstanceIdentifier = config.get('TRANSACC', 'DB_INSTANCE_ID'),
            DBName = config.get('TRANSACC', 'DB_NAME'),
            DBInstanceClass = 'db.t3.micro',
            Engine = 'postgres',
            MasterUsername = config.get('TRANSACC', 'DB_USER'),
            MasterUserPassword = config.get('TRANSACC', 'DB_PASSWORD'),
            Port = int(config.get('TRANSACC', 'DB_PORT')),
            PubliclyAccessible = True,
            VpcSecurityGroupIds = [config.get('VPC', 'SECURITY_GROUP')],
            AllocatedStorage = 15
        )
    print(response)
except aws_rds_conn.exceptions.DBInstanceAlreadyExistsFault as ex:
    print('La instancia ya existe')
except Exception as ex:
    print('Error!', ex)

La instancia ya existe


### Obtener URL hostname de la instancia

In [6]:
try:

    instance = aws_rds_conn.describe_db_instances(DBInstanceIdentifier = config.get('TRANSACC', 'DB_INSTANCE_ID'))
    RDS_HOSTNAME = instance.get('DBInstances')[0].get('Endpoint').get('Address')
    print(RDS_HOSTNAME)

except Exception as ex:
    print('Error!!!', ex)


e-dreams-transactional.cfum2oeu0d6b.us-east-1.rds.amazonaws.com


### Nos conectamos a la base de datos

In [25]:
import sql_queries_e
sql_queries_e.DDL_QUERY

'\nCREATE TABLE IF NOT EXISTS categoria(\n    id_categoria INT PRIMARY KEY,\n    nombre VARCHAR(50) UNIQUE,\n    descripcion VARCHAR(255),\n    estado_categoria BOOLEAN\n);\n\nCREATE TABLE IF NOT EXISTS articulo(\n    id_articulo INT PRIMARY KEY,\n    id_categoria INT REFERENCES categoria(id_categoria),\n    codigo VARCHAR(50),\n    nombre VARCHAR(100),\n    precio_venta NUMERIC(11,2),\n    stock INT,\n    descripcion VARCHAR(255),\n    imagen VARCHAR(20),\n    estado_articulo BOOLEAN\n);\n\nCREATE TABLE IF NOT EXISTS persona(\n    id_persona INT PRIMARY KEY,\n    tipo_persona VARCHAR(20),\n    nombre VARCHAR(100),\n    tipo_documento VARCHAR(20),\n    num_documento VARCHAR(20),\n    direccion VARCHAR(20),\n    telefono VARCHAR(20),\n    email VARCHAR(20)\n);\n\nCREATE TABLE IF NOT EXISTS rol(\n    id_rol INT PRIMARY KEY,\n    nombre VARCHAR(30),\n    descripcion VARCHAR(30),\n    estado_rol BOOLEAN\n);\n\nCREATE TABLE IF NOT EXISTS usuario(\n    id_usuario INT PRIMARY KEY,\n    id_rol

In [26]:
try:
    db_pg_conn = psycopg2.connect(
            database =config.get('TRANSACC', 'DB_NAME'),
            user = config.get('TRANSACC', 'DB_USER'),
            password = config.get('TRANSACC', 'DB_PASSWORD'),
            host = RDS_HOSTNAME,
            port = int(config.get('TRANSACC', 'DB_PORT')),
    )

    cursor = db_pg_conn.cursor()
    cursor.execute(sql_queries_e.DDL_QUERY)
    db_pg_conn.commit()
    print('Base de Datos Creada Exitosamente')

except Exception as ex:
    print('Error!!!', ex)

Base de Datos Creada Exitosamente


### Insertamos datos en la base de datos

In [27]:
def insertData2SQL(data_dict, table_name, driver):


    df_data = pd.DataFrame.from_records(data_dict)

    try:
        response = df_data.to_sql(table_name, driver, index = False, if_exists='append')
        print(f"Se han insertado {response} nuevos registros")
    except Exception as ex:
        print(ex)

In [28]:
driver = f"""postgresql://{config.get('TRANSACC', 'DB_USER')}:{config.get('TRANSACC', 'DB_PASSWORD')}@{RDS_HOSTNAME}:{config.get('TRANSACC', 'DB_PORT')}/{config.get('TRANSACC', 'DB_NAME')}"""

In [29]:
import random
from faker import Faker

faker = Faker()

In [31]:
Cantidad_articulos = 1000
Cantidad_personas = 10
Cantidad_usuarios = 10000
Cantidad_ingresos = 1000
Cantidad_detalle_ingreso = 3000
Cantidad_ventas = 1000
Cantidad_detalle_venta = 3000

In [32]:
categoria = [
     {'id_categoria': 10008, 'nombre': 'comestibles', 'descripcion': 'comida', 'estado_categoria':'TRUE'}, 
     {'id_categoria': 10010, 'nombre': 'AlimentoMascotas', 'descripcion': 'Mascotas', 'estado_categoria':'TRUE'},
     {'id_categoria': 10015, 'nombre': 'Golosinas','descripcion': 'Chocolates, dulces', 'estado_categoria':'TRUE'},
     {'id_categoria': 10018, 'nombre': 'MuebleCaja','descripcion': 'articulos de muebles de caja', 'estado_categoria':'TRUE'},
     {'id_categoria': 10025, 'nombre': 'gorros','descripcion': 'gorros para frio', 'estado_categoria':'FALSE'},
]

#insertamos data en tabla tipo_transaccione 
insertData2SQL(categoria, 'categoria', driver)

Se han insertado 5 nuevos registros


In [37]:
data_articulo = []


for i in range(Cantidad_articulos):

    articulo = {
        'id_articulo' : i,
        'id_categoria': random.choice(categoria)['id_categoria'],
        'codigo': np.random.randint(10000, 99999),
        'nombre': faker.word(),
        'precio_venta':faker.random_number(digits=4, fix_len=True) / 100,
        'stock' : faker.random_number(digits=2),
        'descripcion': faker.text(max_nb_chars=200),
        'imagen': faker.text(max_nb_chars=20),
        'estado_articulo' :faker.random_element(elements=(True, False)),
    }

    data_articulo.append(articulo)    

insertData2SQL(data_articulo,'articulo', driver)

Se han insertado 1000 nuevos registros


In [38]:
data_persona = []

for i in range(Cantidad_personas):
    ClientProfile = faker.profile()
    
    persona = {
        'id_persona' : i,
        'tipo_persona' : faker.random_element(elements=('Juridica', 'Individual')),
        'nombre': ClientProfile['name'],
        'tipo_documento': faker.random_element(elements=('DPI', 'Licencia')),
        'num_documento': ClientProfile['ssn'],
        'direccion': ClientProfile['address'][:20],
        'telefono' : np.random.randint(10000000, 99999999),
        'email': ClientProfile['mail'][:20],
    }

    data_persona.append(persona)

insertData2SQL(data_persona, 'persona', driver)


Se han insertado 10 nuevos registros


In [39]:
rol = [
     {'id_rol': 14586, 'nombre': 'Gerente', 'descripcion': 'Acceso maximo', 'estado_rol':'TRUE'}, 
     {'id_rol': 18568, 'nombre': 'Administrador', 'descripcion': 'Acceso medio', 'estado_rol':'TRUE'},
     {'id_rol': 16578, 'nombre': 'Cliente','descripcion': 'Acceso minimo', 'estado_rol':'TRUE'},
]

#insertamos data en tabla tipo_transaccione 
insertData2SQL(rol, 'rol', driver)

Se han insertado 3 nuevos registros


In [40]:
data_usuario = []


for i in range(Cantidad_usuarios):
    ClientProfile = faker.profile()

    usuario = {
        'id_usuario' : i,
        'id_rol' : random.choice(rol)['id_rol'],
        'nombre': ClientProfile['name'],
        'tipo_documento': faker.random_element(elements=('DPI', 'Licencia')),
        'num_documento': ClientProfile['ssn'],
        'direccion':ClientProfile['address'][:20],
        'telefono' : np.random.randint(10000000, 99999999),
        'email': ClientProfile['mail'][:20],
        'clave': faker.password(length=12, special_chars=True, digits=True, upper_case=True, lower_case=True),
        'estado_usuario': faker.random_element(elements=(True, False)),
    }

    data_usuario.append(usuario)    

insertData2SQL(data_usuario,'usuario', driver)

Se han insertado 1000 nuevos registros


In [51]:
data_ingreso= []


for i in range(Cantidad_ingresos):
    ClientProfile = faker.profile()

    ingreso = {
        'id_ingreso' : i,
        'id_proveedor' : random.choice(data_persona)['id_persona'],
        'id_usuario' : random.choice(data_usuario)['id_usuario'],
        'tipo_comprobante': faker.random_element(elements=('A', 'B')),
        'serie_comprobante': faker.pystr(min_chars=7, max_chars=7),
        'num_comprobante': faker.random_number(digits=8),
        'fecha':faker.date_this_decade(),
        'impuesto' : faker.random_number(digits=3, fix_len=True) / 100,
        'total': faker.random_number(digits=4, fix_len=True) / 100,
        'estado_venta': faker.random_element(elements=(True, False)),
    }

    data_ingreso.append(ingreso)    

insertData2SQL(data_ingreso,'ingreso', driver)

Se han insertado 1000 nuevos registros


In [52]:
data_detalle_ingreso= []


for i in range(Cantidad_detalle_ingreso):
    ClientProfile = faker.profile()

    detalle_ingreso = {
        'id_detalle_ingreso' : i,
        'id_ingreso' : random.choice(data_ingreso)['id_ingreso'],
        'id_articulo' : random.choice(data_articulo)['id_articulo'],
        'cantidad': faker.random_number(digits=3),#aqui tengo duda que es esto
        'precio_unitario': faker.random_number(digits=4, fix_len=True) / 100,
    }

    data_detalle_ingreso.append(detalle_ingreso)    

insertData2SQL(data_detalle_ingreso,'detalle_ingreso', driver)

Se han insertado 1000 nuevos registros


In [43]:
data_venta= []


for i in range(Cantidad_ventas):
    ClientProfile = faker.profile()

    venta = {
        'id_venta' : i,
        'id_cliente' : faker.random_number(digits=6),
        'id_usuario' : random.choice(data_usuario)['id_usuario'],
        'id_persona' : random.choice(data_persona)['id_persona'],
        'tipo_comprobante': faker.random_element(elements=('A', 'B')),#aqui tengo duda que es esto
        'serie_comprobante': faker.pystr(min_chars=7, max_chars=7),
        'num_comprobante': faker.pystr(min_chars=10, max_chars=10),
        'fecha':faker.date_this_decade(),
        'impuesto' : faker.random_number(digits=2, fix_len=True) / 100,
        'total': faker.random_number(digits=2, fix_len=True) / 100,#aqui tengo duda se suma algo?
        'estado_venta': faker.random_element(elements=('A', 'B','C')),#aqui tengo duda ques es esto
    }

    data_venta.append(venta)    

insertData2SQL(data_venta,'venta', driver)

Se han insertado 1000 nuevos registros


In [50]:
data_detalle_venta= []


for i in range(Cantidad_detalle_venta):
    ClientProfile = faker.profile()

    detalle_venta = {
        'id_detalle_venta' : i,
        'id_venta' : random.choice(data_venta)['id_venta'],
        'id_articulo' : random.choice(data_articulo)['id_articulo'],
        'cantidad': faker.random_number(digits=3),#aqui tengo duda que es esto
        'precio': faker.random_number(digits=4, fix_len=True) / 100,
        'descuento': faker.random_number(digits=2, fix_len=True) / 100,
    }

    data_detalle_venta.append(detalle_venta)    

insertData2SQL(data_detalle_venta,'detalle_venta', driver)

Se han insertado 1000 nuevos registros
